<a href="https://colab.research.google.com/github/sepehrX12/Intrinsic-Alignment-Project/blob/main/CCL_LSST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install pyccl

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...

Looking for: ['pyccl']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   1%
conda-forge/noarch    24%[+] 0.3s
conda-forge/linux-64   1%
conda-forge/noarch    24%[+] 0.4s
conda-forge/linux-64   6%
conda-forge/noarch    45%[+] 0.5s
conda-forge/linux-64  11%
conda-forge/noarch    56%[+] 0.6s
conda-forge/linux-64  16%
conda-forge/noarch    67%[+] 0.7s
conda-forge/linux-64  20%
conda-forge/noarch    78%[+] 0.8s
conda-forge/linux-64  25%
conda-forge/noarch   100%conda-forge/noarch                                
[+] 0.9s
conda-forge/linux-64  30%[+] 1.0s
conda-forge/linux-64  39%[+] 1.1s
conda-forge/linux-64  48%[+] 1.2s
conda-forge/linux-64  57%[+] 1.3s
conda-forge/linux-64  76%[+] 1.4s
conda-forge/linux-64 

In [1]:
import pyccl as ccl
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create new Cosmology object with a given set of parameters. This keeps track
# of previously-computed cosmological functions
cosmo = ccl.Cosmology(
    Omega_c=0.27, Omega_b=0.045, h=0.67, sigma8=0.8, n_s=0.96,
    transfer_function='bbks')

# Define a simple binned galaxy number density curve as a function of redshift
z_n = np.linspace(0., 1., 500)
n = np.ones(z_n.shape)

# Create objects to represent tracers of the weak lensing signal with this
# number density (with has_intrinsic_alignment=False)
lens1 = ccl.WeakLensingTracer(cosmo, dndz=(z_n, n))
lens2 = ccl.WeakLensingTracer(cosmo, dndz=(z_n, n))

# Calculate the angular cross-spectrum of the two tracers as a function of ell
ell = np.arange(2, 10)
cls = cosmo.angular_cl(lens1, lens2, ell)
print(cls)

[8.68063555e-09 1.17960668e-08 1.30533061e-08 1.35122646e-08
 1.35757834e-08 1.34276819e-08 1.31622980e-08 1.28319517e-08]
